<h1 align="center">Preparación de datos</h1>
<hr />

## Se importan los datos a usar

In [1]:
import pandas as pd
import re

## Se realiza la normalización de datos

### 1. Se carga el primer dataset

In [2]:
dfdelitos = pd.read_csv("DataCrim_SinProcesar.csv")

### 2. Se usan expresiones regulares para extraer solo el texto, debido a que en algunos campos se encuentran números y/o carácteres anomalos

In [5]:
dfdelitos["Generica"] = dfdelitos["Generica"].str.extract(r'([A-Za-zÑñÁ-Ú-, ]+)')
dfdelitos["Especifica"] = dfdelitos["Especifica"].str.extract(r'([A-Za-zÑñÁ-Ú-, ]+)')
dfdelitos["Modalidad"] = dfdelitos["Modalidad"].str.extract(r'([A-Za-zÑñÁ-Ú-, ]+)')
dfdelitos["Departamento"] = dfdelitos["Departamento"].str.extract(r'([A-Za-zÑñÁ-Ú-, ]+)')
dfdelitos["Provincia"] = dfdelitos["Provincia"].str.extract(r'([A-Za-zÑñÁ-Ú-, ]+)')
dfdelitos["Distrito"] = dfdelitos["Distrito"].str.extract(r'([A-Za-zÑñÁ-Ú-, ]+)')

### 3. Para los textos, se hace un strip para eliminar los espacios al principio y al final del texto

In [6]:
dfdelitos["Generica"] = dfdelitos["Generica"].str.strip()
dfdelitos["Especifica"] = dfdelitos["Especifica"].str.strip()
dfdelitos["Modalidad"] = dfdelitos["Modalidad"].str.strip()
dfdelitos["Departamento"] = dfdelitos["Departamento"].str.strip()
dfdelitos["Provincia"] = dfdelitos["Provincia"].str.strip()
dfdelitos["Distrito"] = dfdelitos["Distrito"].str.strip()

### 4. Se actualiza el csv con los nuevos valores

In [7]:
dfdelitos.to_csv("DataCrim.csv", index=False)

### 5. Se carga el segundo dataset

In [8]:
dfRobosLima = pd.read_csv("crimenes.csv", sep = ';', encoding = "ISO-8859-1")

### 6. Se recorre el dataset para derle un correcto formato a la fecha, debido a que en el dataset se encuentra con un formate de "DD/MM/YY" y la base de datos requiere un formato de "YYYY/MM/DD"

In [9]:
arr = []
year = ''
month = ''
day = ''
for i in range(len(dfRobosLima["FECHA"])):
    
    if(str(dfRobosLima["FECHA"][i]) == 'nan'):
        arr.append('1900/01/01')
        
    elif(('/' in dfRobosLima["FECHA"][i]) or ('-' in dfRobosLima["FECHA"][i])):
        sp = re.findall(r"([0-9]+)", dfRobosLima["FECHA"][i])
        year = str(sp[2]) if len(str(sp[2])) > 2 else '20'+str(sp[2])
        year = year.replace('0217','2017')
        year = year.replace('2107','2017')
        if int(sp[1]) <= 12:
            month = str(sp[1])
            day = str(sp[0])
        else:
            month = str(sp[0])
            day = str(sp[1])
#         month = str(sp[1] if int(sp[1]) <= 12 else sp[0])
#         day = str(sp[0] if int(sp[0]) <= 12 else sp[1])
        arr.append(year + '/' + month + '/' + day)
        
    else:
        j = i
        while True:
            j+=1
            if('/' in dfRobosLima["FECHA"][j] or '-' in dfRobosLima["FECHA"][j]):
                sp = re.findall(r"([0-9]+)", dfRobosLima["FECHA"][j])
                break
        arr.append((str(sp[2]) if len(str(sp[2])) > 2 else '20'+str(sp[2])) + '/' + 
                   str(sp[1] if int(sp[1]) <= 12 else sp[0]) + '/' + 
                   str(dfRobosLima["FECHA"][i]))
        
dfRobosLima["FECHA"] = pd.DataFrame({"FECHA":arr})

### 7. Para los valores que esten nulos, se reemplaza pur el texto de "No especifica"

In [10]:
text = "No Especifica"
dfRobosLima.loc[dfRobosLima["EDAD VICTIMA"].isnull(), "EDAD VICTIMA"] = text
dfRobosLima.loc[dfRobosLima["EDAD DELINCUENTE"].isnull(), "EDAD DELINCUENTE"] = text
dfRobosLima.loc[dfRobosLima["EDAD VICTIMA"].str.contains('\(n'), "EDAD VICTIMA"] = text
dfRobosLima.loc[dfRobosLima["EDAD DELINCUENTE"].str.contains('\(n'), "EDAD DELINCUENTE"] = text
dfRobosLima.loc[dfRobosLima["OCUPACIÓN"].isnull(), "OCUPACIÓN"] = text
dfRobosLima.loc[dfRobosLima["GENERO DELINCUENTE"].isnull(), "GENERO DELINCUENTE"] = text
dfRobosLima.loc[dfRobosLima["GENERO VICTIMA"].isnull(), "GENERO VICTIMA"] = text
dfRobosLima.loc[dfRobosLima["GENERO DELINCUENTE"].str.contains('\(n'), "GENERO DELINCUENTE"] = text
dfRobosLima.loc[dfRobosLima["GENERO VICTIMA"].str.contains('\(n'), "GENERO VICTIMA"] = text

### 8. Para los campos de tipos de arma que esten nulos, se especifica el valor de otros

In [11]:
dfRobosLima.loc[dfRobosLima["TIPO DE ARMA"].isnull(), "TIPO DE ARMA"] = "Otros"

### 9. Para los campos de heridos o muertos que esten nulos, se inserta el valor de 0

In [12]:
dfRobosLima.loc[dfRobosLima["HERIDOS"].isnull(), "HERIDOS"] = 0
dfRobosLima.loc[dfRobosLima["MUERTOS"].isnull(), "MUERTOS"] = 0

### 10. Para el campo distrito, se hace un upper para que se inserte en mayusculas

In [13]:
dfRobosLima["DISTRITO"] = dfRobosLima["DISTRITO"].str.upper()

### 11. Se reemplazan los valores acentuados con su aquivalente sin acentuar

In [14]:
dfRobosLima["DISTRITO"] = dfRobosLima["DISTRITO"].str.replace('Í','I')
dfRobosLima["DISTRITO"] = dfRobosLima["DISTRITO"].str.replace('Ú', 'U')
dfRobosLima["DISTRITO"] = dfRobosLima["DISTRITO"].str.replace('Ó', 'O')

### 12. Se eliminan valores que contengan datos anomalos

In [15]:
dfRobosLima = dfRobosLima.drop(dfRobosLima[dfRobosLima["GENERO VICTIMA"].str.contains('/')].index)
dfRobosLima = dfRobosLima.drop(dfRobosLima[dfRobosLima["GENERO DELINCUENTE"].str.contains('/')].index)
dfRobosLima = dfRobosLima.drop(dfRobosLima[dfRobosLima["EDAD VICTIMA"].str.contains('/')].index)
dfRobosLima = dfRobosLima.drop(dfRobosLima[dfRobosLima["EDAD DELINCUENTE"].str.contains('/')].index)

### 13. Se almacena el DataFrame en un csv

In [16]:
dfRobosLima.to_csv("CrimenesLima.csv", index=False)